In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

## add your path as a variable below aman_path
aman_path = '/content/gdrive/My Drive/School/Undergrad/Fall 2021/CS 490/Group Project/Code'
# shreyas_path = '/content/gdrive/My Drive/CS490'

## cd into your path instead of aman_path. Don't delete, just comment out.
%cd -q $aman_path
# %cd -q $shreyas_path

Mounted at /content/gdrive/


In [ ]:
from glob import glob
import tensorflow.keras as keras
import tensorflow as tf
import pickle
import pandas as pd

!pip install memory_profiler
from memory_profiler import profile

!pip install treelib
import treelib

!pip install bintrees
import bintrees

import sys
## add your path as a variable instead of aman_path. Don't delete, just comment out and add yours below.
sys.path.append(f'{aman_path}/structures')
# sys.path.append(f'{shreyas_path}/structures')

from b_tree import Item
import numpy as np

from timeit import default_timer as timer
import copy

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=1cd0b6ed006f540e5ff36a5c796824dcba9ca7c4f347d248df029c548840fd22
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
Successfully built memory-profiler
  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18386 sha256=91cbf4b10dd04cdd9fe51dac8fe5221085bc0d25c2b0ee2dd157560604fab2d5
  Stored in directory: /root/.cache/pip/wheels/89/be/94/2c6d949ce599d1443426d83ba4dc93cd35c0f4638260930a53
Successfully built treelib
     |████████████████████████████████| 108 kB 5.2 MB/s 
  Created wheel for bintrees: filename=bintrees-2.2.0-cp37-cp37m-linux_x86_64.whl size=183847 sha256=a2393ce27fc659c507df61d228580bdbb016691e9bb1071a6677064104147b3b
  Stored in directory: /root/.cache/pip/wheels/f3/2d/47/a7577837f4a2bbe902555a41f9229291cf2d06c79bd897d046
Successfully built bintrees


In [ ]:
IS_DATA_STD = True

if IS_DATA_STD:
  std = 'Standardized'
else:
  std = 'Unstandardized'

# Load Models

In [69]:
mod_paths = glob(f'./models/{std}/*.dat')
ann_obj_paths = glob(f'./models/{std}/*/')
mods = {}
for mod_path in mod_paths:
    mod_name = mod_path.split('/')[-1].split('.')[0]
    print(f'Loading pickle models for {mod_name}...')
    with open(mod_path, 'rb') as f:
        data = pickle.load(f)
        mods[mod_name] = data

for obj_path in ann_obj_paths:
    data_name = obj_path.split('/')[-2].split('_')[-1]
    mod_name = '_'.join(obj_path.split('/')[-2].split('_')[:-1])
    print(f'Loading keras {mod_name} model for {data_name}...')
    mod = keras.models.load_model(obj_path)
    mods['ann'][data_name][mod_name]['mod'] = mod  

Loading pickle models for bt...
Loading pickle models for avl...
Loading pickle models for rbt...
Loading pickle models for lr...
Loading pickle models for ann...
Loading keras ann_(32, 32)_true_relu model for amzn...
Loading keras ann_(32, 32)_true_relu model for binomial...
Loading keras ann_(32, 32)_true_relu model for exponential...
Loading keras ann_(32, 32)_true_relu model for random...
Loading keras ann_(32, 32)_true_relu model for fb...
Loading keras ann_(32, 32)_true_relu model for lognormal...
Loading keras ann_(32, 32)_true_relu model for poisson...


# Load Data

In [ ]:
def get_data(path):
    with open(path, 'rb') as f:
        contents = pickle.load(f)
    return contents['data'], contents['memory']



In [75]:
data_paths = glob(f'../Data/{std}/*.dat')
data = {}
for data_path in data_paths:
  data_name = data_path.split('/')[-1].split('.')[0]
  data[data_name] = get_data(data_path)


# Benchmark

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

!pip install pympler
from pympler import asizeof

     |████████████████████████████████| 178 kB 5.1 MB/s 
  Created wheel for pympler: filename=Pympler-0.9-py3-none-any.whl size=164824 sha256=a0333d1a9f3fb2738e5a17806eea30f5ddb12611448c76f774ebe072315f93a8
  Stored in directory: /root/.cache/pip/wheels/1a/f3/d8/35d5614ea4ddd295ffb9372a5f2f9570d9593d1ea4be33ec6d
Successfully built pympler


In [70]:
def predict(mod, data, name=None, labs=None):
  if name == 'bt':
    if labs is None:
      raise Exception('Must provided labels if predicting using b-tree!')
    data = data[:, 0]
    preds = []
    for value, lab in zip(data, labs):
      item = Item(value, lab)
      preds.append(mod.predict(item))
    return np.array(preds).reshape(-1, 1)

  elif name in ['avl', 'rbt']:
    data = data[:, 0]
    preds = []
    for value in data:
      preds.append(mod.get(value))
    return np.array(preds).reshape(-1, 1)
  
  else:
    try:
      return mod.predict(data)
    except(ValueError):
      data = np.asarray(data).astype('float32')
      return mod.predict(data)
  
def get_stats(mods, data, folds=5):
  mod_stats = {mod: {dist: {} for dist in mods[list(mods.keys())[0]].keys()} for mod in mods}
  for mod in mods:
    for dist in data: 
      print(f'Predicting with model {mod} for distribution {dist}...')
      if mod == 'ann':
        for ann in mods[mod][dist]:
          mod_stats[mod][dist][ann] = {}
          for it in range(folds):
            start = timer()
            pred = predict(mods[mod][dist][ann]['mod'], data[dist][0].reshape(-1, 1), name=mod, labs=data[dist][1])
            end = timer()

            true = data[dist][1].reshape(-1, 1)
            pred_srs = pd.Series(pred[:, 0])
            if pred_srs.isna().sum().any():
              pred = pred[pred_srs[~pred_srs.isna()].index,:]
              true = true[pred_srs[~pred_srs.isna()].index,:]

            mse = mean_squared_error(true, pred)
            mae = mean_absolute_error(true, pred)
            size = asizeof.asizeof(mods[mod][dist][ann]['mod'])

            try:
              mod_stats[mod][dist][ann]['predict_time'].append(end - start)
              mod_stats[mod][dist][ann]['mse'].append(mse)
              mod_stats[mod][dist][ann]['mae'].append(mae)
              mod_stats[mod][dist][ann]['mod_size'].append(size)

            except KeyError:
              mod_stats[mod][dist][ann]['predict_time'] = [(end - start)]
              mod_stats[mod][dist][ann]['mse'] = [mse]
              mod_stats[mod][dist][ann]['mae'] = [mae]
              mod_stats[mod][dist][ann]['mod_size'] = [size]
            
      else:  
        for it in range(folds):  
          start = timer()
          pred = predict(mods[mod][dist]['mod'], data[dist][0].reshape(-1, 1), name=mod, labs=data[dist][1])
          end = timer()

          true = data[dist][1].reshape(-1, 1)
          pred_srs = pd.Series(pred[:, 0])
          if pred_srs.isna().sum().any():
            pred = pred[pred_srs[~pred_srs.isna()].index,:]
            true = true[pred_srs[~pred_srs.isna()].index,:]

          mse = mean_squared_error(true, pred)
          mae = mean_absolute_error(true, pred)
          size = asizeof.asizeof(mods[mod][dist]['mod'])

          try:
            mod_stats[mod][dist]['predict_time'].append(end - start)
            mod_stats[mod][dist]['mse'].append(mse)
            mod_stats[mod][dist]['mae'].append(mae)
            mod_stats[mod][dist]['mod_size'].append(size)

          except KeyError:
            mod_stats[mod][dist]['predict_time'] = [(end - start)]
            mod_stats[mod][dist]['mse'] = [mse]
            mod_stats[mod][dist]['mae'] = [mae]
            mod_stats[mod][dist]['mod_size'] = [size]
  return mod_stats




In [76]:
stats = get_stats(mods, data)

Predicting with model bt for distribution fb...
Predicting with model bt for distribution amzn...
Predicting with model bt for distribution random...
Predicting with model bt for distribution binomial...
Predicting with model bt for distribution poisson...
Predicting with model bt for distribution exponential...
Predicting with model bt for distribution lognormal...
Predicting with model avl for distribution fb...
Predicting with model avl for distribution amzn...
Predicting with model avl for distribution random...
Predicting with model avl for distribution binomial...
Predicting with model avl for distribution poisson...
Predicting with model avl for distribution exponential...
Predicting with model avl for distribution lognormal...
Predicting with model rbt for distribution fb...
Predicting with model rbt for distribution amzn...
Predicting with model rbt for distribution random...
Predicting with model rbt for distribution binomial...
Predicting with model rbt for distribution pois

In [77]:
with open(f'./benchmarks/{std.lower()}_benchmarks.dat', 'wb') as f:
  pickle.dump(stats, f)
